# QSM Reconstruction

In [ ]:
!qsmxt bids out \
    --premade 'body' \
    --do_qsm \
    --do_t2starmap \
    --do_r2starmap \
    --do_swi \
    --mask_erosions 3 0 \
    --threshold_value 25.0 \
    --slurm a_barth general \
    --auto_yes

# Imports

In [1]:
import os
import glob
import nibabel as nib
import nilearn.image
import numpy as np
import enum
import json
from scipy.ndimage import label
import cv2
import scipy
from scipy.ndimage import binary_dilation, binary_erosion, generate_binary_structure, label, center_of_mass

# Locate data

In [2]:
bids_dir = "bids"

In [3]:
session_dirs = []
for json_path in sorted(glob.glob(os.path.join(bids_dir, "sub*", "ses*", "anat", "*echo-01*mag*json"))):
    with open(json_path, 'r') as json_file:
        json_data = json.load(json_file)
        if json_data['ProtocolName'] in ["wip_iSWI_fl3d_vibe_TRY THIS ONE"]:#, "wip_iSWI_fl3d_vibe", "wip_iSWI_fl3d_vibe_TRY THIS ONE"]:
            session_dirs.append(os.sep.join(os.path.split(json_path)[0].split(os.sep)[:-1]))

# remove all sessions with 'sub-z0449294' in the path
session_dirs = sorted(set([s for s in session_dirs if 'sub-z0449294' not in s]))
print(f"{len(session_dirs)} sessions found.")

25 sessions found.


In [4]:
extra_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "*.nii*")) for session_dir in session_dirs), []))

qsm_files2 = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "*real.nii*")) for session_dir in session_dirs), []))
qsm_files = [x for x in sorted(glob.glob(os.path.join("out/qsm/*.nii"))) if 'sub-z0449294' not in x]
t2s_files = [x for x in sorted(glob.glob(os.path.join("out/t2s/*.nii"))) if 'sub-z0449294' not in x]
r2s_files = [x for x in sorted(glob.glob(os.path.join("out/r2s/*.nii"))) if 'sub-z0449294' not in x]
swi_files = [x for x in sorted(glob.glob(os.path.join("out/swi/*swi.nii"))) if 'sub-z0449294' not in x]
mag_files = [extra_file for extra_file in extra_files if os.path.split(extra_file)[1] == 'magnitude_combined.nii']
fmap_files = [extra_file for extra_file in extra_files if os.path.split(extra_file)[1] == 'B0.nii']

gre_seg_clean_files = sorted([extra_file for extra_file in extra_files if all(pattern in extra_file for pattern in ['segmentation_clean.', 'tgvqsmjl'])])
t1_files = sorted([extra_file for extra_file in extra_files if 'T1w_resliced' in extra_file])

# Missing CT for sub-z0449294 
ct_files = sorted([extra_file for extra_file in extra_files if 'resliced' in extra_file and 'T1w' not in extra_file and 'segmentation' not in extra_file and 'downsampled' not in extra_file])
ct_seg_files = sorted([ct_file.replace(".nii", "_segmentation.nii") for ct_file in ct_files if os.path.exists(ct_file.replace(".nii", "_segmentation.nii"))])
gre_seg_files = ct_seg_files
ct_seg_clean_files = sorted([ct_file.replace(".nii", "_clean.nii") for ct_file in ct_seg_files if os.path.exists(ct_file.replace(".nii", "_clean.nii"))])

print(f"{len(fmap_files)} field maps found.")
print(f"{len(ct_files)} CT images found.")
print(f"{len(ct_seg_files)} raw CT segmentations found.")
print(f"{len(ct_seg_clean_files)} clean CT segmentations found.")
print(f"{len(gre_seg_files)} raw GRE segmentations found.")
print(f"{len(gre_seg_clean_files)} clean GRE segmentations found.")
print(f"{len(qsm_files)} QSM images found.")
print(f"{len(qsm_files2)} QSM (2) images found.")
print(f"{len(mag_files)} magnitude images found.")
print(f"{len(t2s_files)} T2* maps found.")
print(f"{len(r2s_files)} R2* maps found.")
print(f"{len(swi_files)} SWI maps found.")
print(f"{len(t1_files)} T1w files found.")

25 field maps found.
25 CT images found.
25 raw CT segmentations found.
25 clean CT segmentations found.
25 raw GRE segmentations found.
25 clean GRE segmentations found.
25 QSM images found.
25 QSM (2) images found.
25 magnitude images found.
25 T2* maps found.
25 R2* maps found.
25 SWI maps found.
25 T1w files found.


# Clean up segmentations

## CT

In [10]:
class SegType(enum.Enum):
    NO_LABEL = 0
    PROSTATE = 1
    GOLD_SEED = 2
    CALCIFICATION = 3

def create_spherical_mask(center, shape, radius):
    # Create an array filled with indices
    indices = np.indices(shape).transpose((1, 2, 3, 0))
    # Calculate the distance of each index from the center
    distances = np.linalg.norm(indices - center, axis=-1)
    # Create a mask where the distance is less than or equal to the radius
    mask = distances <= radius
    return mask

for i in range(len(qsm_files)):
    # load files
    qsm_nii = nib.load(qsm_files[i])
    seg_nii = nib.load(ct_seg_files[i])

    # get image data
    qsm = qsm_nii.get_fdata()
    seg = np.array(seg_nii.get_fdata(), dtype=np.uint8)

    # get prostate values
    prostate_values = qsm[seg == SegType.PROSTATE.value]

    stdevs = 2.5

    # automated gold seed mask
    seed_mask_labelled, num_labels = label(
        input=seg == SegType.GOLD_SEED.value,
        structure=np.ones((3, 3, 3))
    )
    seed_mask = np.zeros(seg.shape, dtype=np.uint8)
    for seed_id in range(1, num_labels + 1):
        seed_i_mask = seed_mask_labelled == seed_id
        seed_i_mean = qsm[seed_i_mask].mean()

        # get centre voxel of seed mask
        centre_voxel = center_of_mass(seed_i_mask)
        seed_i_mask = create_spherical_mask(centre_voxel, seed_i_mask.shape, radius=2.5)

        '''
        if seed_i_mean > prostate_values.mean() + 1*prostate_values.std():
            seed_i_mask = np.logical_and(seed_i_mask, qsm < prostate_values.mean() - stdevs*prostate_values.std())
            seed_i_mask = np.logical_and(binary_dilation(seed_i_mask), (qsm < np.mean(prostate_values) - stdevs*np.std(prostate_values)))
            #seed_i_mask = np.logical_and(seed_i_mask, qsm > prostate_values.mean() + stdevs*prostate_values.std())
            #seed_i_mask = np.logical_and(binary_dilation(seed_i_mask), (qsm > np.mean(prostate_values) + stdevs*np.std(prostate_values)))
        else:
            seed_i_mask = np.logical_and(seed_i_mask, qsm < prostate_values.mean() - stdevs*prostate_values.std())
            seed_i_mask = np.logical_and(binary_dilation(seed_i_mask), (qsm < np.mean(prostate_values) - stdevs*np.std(prostate_values)))
        '''
        seed_mask = np.logical_or(seed_mask, seed_i_mask)
    structure = generate_binary_structure(3, 1) # 3 for 3D image, 2 for full connectivity
    labeled_mask, num_features = label(seed_mask, structure)
    for idx in range(1, num_features + 1):
        if np.sum(labeled_mask == idx) == 1:
            seed_mask[labeled_mask == idx] = 0
    seed_mask = np.logical_and(seed_mask, seg == SegType.GOLD_SEED.value)

    # automated calcification mask
    calc_mask = np.array(seg == SegType.CALCIFICATION.value, dtype=np.uint8)
    calc_mask = np.logical_and(calc_mask, qsm < prostate_values.mean() - stdevs*prostate_values.std())
    calc_mask = np.logical_and(binary_dilation(calc_mask), (qsm < np.mean(prostate_values) - stdevs*np.std(prostate_values)))
    structure = generate_binary_structure(3, 1) # 3 for 3D image, 2 for full connectivity
    labeled_mask, num_features = label(calc_mask, structure)
    for idx in range(1, num_features + 1):
        if np.sum(labeled_mask == idx) == 1:
            calc_mask[labeled_mask == idx] = 0
    calc_mask = np.logical_and(calc_mask, seg == SegType.CALCIFICATION.value)

    # setup final values
    seg[seg == SegType.GOLD_SEED.value] = 0
    seg[seg == SegType.CALCIFICATION.value] = 0
    seg[seg == SegType.PROSTATE.value] = 0

    seg[seed_mask] = 1
    seg[calc_mask] = 2
    
    # save result using original file extension
    filename = f"{qsm_files[i].replace('.nii', '_segmentation_clean.nii')}"
    filename = os.path.join(os.path.split(ct_seg_files[i])[0], os.path.split(filename)[1])
    nib.save(nib.Nifti1Image(seg, header=seg_nii.header, affine=seg_nii.affine), filename=filename)
    print(f"Saved {filename}")

Saved bids/sub-z0002292/ses-20240318/extra_data/sub-z0002292_ses-20240318_acq-wipiswifl3dvibetrythisone_run-01_echo-01_part-phase_MEGRE_scaled_tgvqsmjl_twopass_average_ref_segmentation_clean.nii
Saved bids/sub-z0163277/ses-20240123/extra_data/sub-z0163277_ses-20240123_acq-wipiswifl3dvibetrythisone_run-01_echo-01_part-phase_MEGRE_scaled_tgvqsmjl_twopass_average_ref_segmentation_clean.nii
Saved bids/sub-z0206923/ses-20231003/extra_data/sub-z0206923_ses-20231003_acq-wipiswifl3dvibetrythisone_run-01_echo-01_part-phase_MEGRE_scaled_tgvqsmjl_twopass_average_ref_segmentation_clean.nii
Saved bids/sub-z0232771/ses-20240228/extra_data/sub-z0232771_ses-20240228_acq-wipiswifl3dvibetrythisone_run-01_echo-01_part-phase_MEGRE_scaled_tgvqsmjl_twopass_average_ref_segmentation_clean.nii
Saved bids/sub-z0347933/ses-20240129/extra_data/sub-z0347933_ses-20240129_acq-wipiswifl3dvibetrythisone_run-01_echo-01_part-phase_MEGRE_scaled_tgvqsmjl_twopass_average_ref_segmentation_clean.nii
Saved bids/sub-z0381949/s

## MRI

In [17]:
class SegType(enum.Enum):
    NO_LABEL = 0
    PROSTATE = 1
    GOLD_SEED = 2
    CALCIFICATION = 3

for i in range(len(qsm_files)):
    # load files
    ct_nii = nib.load(ct_files[i])
    seg_nii = nib.load(ct_seg_files[i])

    # get image data
    ct = ct_nii.get_fdata()
    seg = np.array(seg_nii.get_fdata(), dtype=np.uint8)

    # get prostate values
    prostate_values = ct[seg == SegType.PROSTATE.value]

    # seed mask
    seed_mask = np.logical_and(seg == SegType.GOLD_SEED.value, ct > prostate_values.mean() + 4*prostate_values.std())
    seed_mask = np.logical_and((ct > np.mean(prostate_values) + 4*np.std(prostate_values)), binary_dilation(seed_mask))
    structure = generate_binary_structure(3, 1)
    labeled_mask, num_features = label(seed_mask, structure)
    for idx in range(1, num_features + 1):
        if np.sum(labeled_mask == idx) == 1:
            seed_mask[labeled_mask == idx] = 0
    seed_mask = np.logical_and(seed_mask, seg == SegType.GOLD_SEED.value)

    # calcification mask
    calc_mask = np.logical_and(seg != SegType.GOLD_SEED, ct > np.mean(prostate_values) + 4*np.std(prostate_values))
    calc_mask = np.logical_and((ct > np.mean(prostate_values) + 4*np.std(prostate_values)), binary_dilation(calc_mask))
    calc_mask = np.logical_and((ct > np.mean(prostate_values) + 4*np.std(prostate_values)), binary_dilation(calc_mask))
    structure = generate_binary_structure(3, 1)
    labeled_mask, num_features = label(calc_mask, structure)
    for idx in range(1, num_features + 1):
        if np.sum(labeled_mask == idx) == 1:
            calc_mask[labeled_mask == idx] = 0
    calc_mask = np.logical_and(calc_mask, seg == SegType.CALCIFICATION.value)
    
    # setup final values
    seg[seg == SegType.GOLD_SEED.value] = 0
    seg[seg == SegType.CALCIFICATION.value] = 0
    seg[seg == SegType.PROSTATE.value] = 0

    seg[seed_mask] = 1
    seg[calc_mask] = 2
    
    # save result using original file extension
    filename = f"{ct_seg_files[i].replace('.nii', '_clean.nii')}"
    nib.save(nib.Nifti1Image(seg, header=seg_nii.header, affine=seg_nii.affine), filename=filename)
    print(f"Saved {filename}")

Saved bids/sub-z0002292/ses-20240318/extra_data/series-6_16_Pelvis_16_Pelvis_20240318092750_6_resliced_segmentation_clean.nii
Saved bids/sub-z0163277/ses-20240123/extra_data/series-5_16_Pelvis_16_Pelvis_20240123085135_5_resliced_segmentation_clean.nii
Saved bids/sub-z0206923/ses-20231003/extra_data/series-5_NA_20231003083739_5_resliced_segmentation_clean.nii
Saved bids/sub-z0232771/ses-20240228/extra_data/series-5_16_Pelvis_16_Pelvis_20240228094427_5_resliced_segmentation_clean.nii
Saved bids/sub-z0347933/ses-20240129/extra_data/series-4_16_Pelvis_16_Pelvis_20240129121046_4_resliced_segmentation_clean.nii
Saved bids/sub-z0381949/ses-20231005/extra_data/series-5_NA_20231005075647_5_resliced_segmentation_clean.nii
Saved bids/sub-z0742379/ses-20240124/extra_data/series-6_16_Pelvis_16_Pelvis_20240124093902_6_resliced_segmentation_clean.nii
Saved bids/sub-z0747844/ses-20240304/extra_data/series-5_16_Pelvis_16_Pelvis_20240304114712_5_resliced_segmentation_clean.nii
Saved bids/sub-z1318033/se

## Field maps

In [18]:
class SegType(enum.Enum):
    NO_LABEL = 0
    PROSTATE = 1
    GOLD_SEED = 2
    CALCIFICATION = 3

def create_spherical_mask(center, shape, radius):
    # Create an array filled with indices
    indices = np.indices(shape).transpose((1, 2, 3, 0))
    # Calculate the distance of each index from the center
    distances = np.linalg.norm(indices - center, axis=-1)
    # Create a mask where the distance is less than or equal to the radius
    mask = distances <= radius
    return mask

for i in range(len(fmap_files)):
    # load files
    fmap_nii = nib.load(fmap_files[i])
    seg_nii = nib.load(ct_seg_files[i])
    qsm_nii = nib.load(qsm_files[i])

    # get image data
    fmap = fmap_nii.get_fdata()
    qsm = qsm_nii.get_fdata()
    seg = np.array(seg_nii.get_fdata(), dtype=np.uint8)

    # get prostate values
    prostate_values_fmap = fmap[seg == SegType.PROSTATE.value]
    prostate_values_qsm = qsm[seg == SegType.PROSTATE.value]
    stdevs = 2.5

    # automated gold seed mask
    seed_mask_labelled, num_labels = label(
        input=seg == SegType.GOLD_SEED.value,
        structure=np.ones((3, 3, 3))
    )
    seed_mask = np.zeros(seg.shape, dtype=np.uint8)
    for seed_id in range(1, num_labels + 1):
        seed_i_mask = seed_mask_labelled == seed_id
        #seed_i_mean = fmap[seed_i_mask].mean()
        seed_i_mask = np.logical_and(seed_i_mask, fmap - fmap[seed_i_mask].mean() > 1*fmap[seed_i_mask].std())

        # get centre voxel of seed mask
        centre_voxel = center_of_mass(seed_i_mask)
        seed_i_mask = create_spherical_mask(centre_voxel, seed_i_mask.shape, radius=2.5)

        seed_mask = np.logical_or(seed_mask, seed_i_mask)
    structure = generate_binary_structure(3, 1) # 3 for 3D image, 2 for full connectivity
    labeled_mask, num_features = label(seed_mask, structure)
    for idx in range(1, num_features + 1):
        if np.sum(labeled_mask == idx) == 1:
            seed_mask[labeled_mask == idx] = 0
    seed_mask = np.logical_and(seed_mask, seg == SegType.GOLD_SEED.value)

    # automated calcification mask
    calc_mask = np.array(seg == SegType.CALCIFICATION.value, dtype=np.uint8)
    calc_mask = np.logical_and(calc_mask, qsm < prostate_values_qsm.mean() - stdevs*prostate_values_qsm.std())
    calc_mask = np.logical_and(binary_dilation(calc_mask), (qsm < np.mean(prostate_values_qsm) - stdevs*np.std(prostate_values_qsm)))
    structure = generate_binary_structure(3, 1) # 3 for 3D image, 2 for full connectivity
    labeled_mask, num_features = label(calc_mask, structure)
    for idx in range(1, num_features + 1):
        if np.sum(labeled_mask == idx) == 1:
            calc_mask[labeled_mask == idx] = 0
    calc_mask = np.logical_and(calc_mask, seg == SegType.CALCIFICATION.value)

    # setup final values
    seg[seg == SegType.GOLD_SEED.value] = 0
    seg[seg == SegType.CALCIFICATION.value] = 0
    seg[seg == SegType.PROSTATE.value] = 0

    seg[seed_mask] = 1
    seg[calc_mask] = 2
    
    # save result using original file extension
    filename = f"{qsm_files[i].replace('.nii', '_segmentation_clean.nii')}"
    filename = os.path.join(os.path.split(ct_seg_files[i])[0], os.path.split(filename)[1])
    nib.save(nib.Nifti1Image(seg, header=seg_nii.header, affine=seg_nii.affine), filename=filename)
    print(f"Saved {filename}")

Saved bids/sub-z0002292/ses-20240318/extra_data/sub-z0002292_ses-20240318_acq-wipiswifl3dvibetrythisone_run-01_echo-01_part-phase_MEGRE_scaled_tgvqsmjl_twopass_average_ref_segmentation_clean.nii
Saved bids/sub-z0163277/ses-20240123/extra_data/sub-z0163277_ses-20240123_acq-wipiswifl3dvibetrythisone_run-01_echo-01_part-phase_MEGRE_scaled_tgvqsmjl_twopass_average_ref_segmentation_clean.nii
Saved bids/sub-z0206923/ses-20231003/extra_data/sub-z0206923_ses-20231003_acq-wipiswifl3dvibetrythisone_run-01_echo-01_part-phase_MEGRE_scaled_tgvqsmjl_twopass_average_ref_segmentation_clean.nii
Saved bids/sub-z0232771/ses-20240228/extra_data/sub-z0232771_ses-20240228_acq-wipiswifl3dvibetrythisone_run-01_echo-01_part-phase_MEGRE_scaled_tgvqsmjl_twopass_average_ref_segmentation_clean.nii
Saved bids/sub-z0347933/ses-20240129/extra_data/sub-z0347933_ses-20240129_acq-wipiswifl3dvibetrythisone_run-01_echo-01_part-phase_MEGRE_scaled_tgvqsmjl_twopass_average_ref_segmentation_clean.nii
Saved bids/sub-z0381949/s

# Generate field maps

This step required some cropping because of high-intensity regions in the magnitude that seem to ruin the field maps generated by ROMEO. Might be a bug for Korbinian to fix.

In [15]:
for session_dir in session_dirs:
    if 'sub-z3220308/ses-20231009' in session_dir:
        print(session_dir)
        
        mag_images = sorted(glob.glob(os.path.join(session_dir, "anat", "*run-01*part-mag*nii*")))
        phs_images = sorted(glob.glob(os.path.join(session_dir, "anat", "*run-01*part-phase*nii*")))

        mag_4d = np.stack([nib.load(f).get_fdata() for f in mag_images], -1)
        phs_4d = np.stack([nib.load(f).get_fdata() for f in phs_images], -1)

        mag_4d_cropped = mag_4d[:,40:150,:,:]
        phs_4d_cropped = phs_4d[:,40:150,:,:]

        phs_4d_cropped = np.array(np.interp(phs_4d_cropped, (phs_4d_cropped.min(), phs_4d_cropped.max()), (-np.pi, +np.pi)), dtype=phs_4d_cropped.dtype)

        mag_nii = nib.load(mag_images[0])
        phs_nii = nib.load(phs_images[0])

        nib.save(nib.Nifti1Image(dataobj=mag_4d_cropped, header=mag_nii.header, affine=mag_nii.affine), "mag.nii")
        nib.save(nib.Nifti1Image(dataobj=phs_4d_cropped, header=phs_nii.header, affine=phs_nii.affine), "phs.nii")

        !~/neurodesktop-storage/repos/qsmxt/qsmxt/interfaces/nipype_interface_romeo.py --phase phs.nii --magnitude mag.nii --TEs 3.31 5.45 7.590000000000001 9.73 11.870000000000001 14.01 16.150000000000002

        B0_data = nib.load("phs_B0.nii").get_fdata()
        B0_uncropped = np.zeros(phs_4d.shape[:-1], dtype=B0_data.dtype)
        B0_uncropped[:, 40:150, :] = B0_data
        B0_uncropped_nii = nib.Nifti1Image(dataobj=B0_uncropped, affine=phs_nii.affine, header=phs_nii.header)
        nib.save(B0_uncropped_nii, os.path.join(session_dir, "extra_data", "B0.nii"))

bids/sub-z3220308/ses-20231009


240404-11:36:15,51 nipype.interface INFO:
	 stdout 2024-04-04T11:36:15.050677: 46.045366 seconds (40.49 M allocations: 3.224 GiB, 7.20% gc time, 89.82% compilation time)
240404-11:36:15,123 nipype.interface INFO:
	 stdout 2024-04-04T11:36:15.122826:0
Processing complete. Unwrapped phase and B0 map generated.
Unwrapped Phase Path(s): /home/ashley/neurodesktop-storage/data/prostate/2024-prostate/phs_romeo-unwrapped.nii
B0 Map Path: /home/ashley/neurodesktop-storage/data/prostate/2024-prostate/phs_B0.nii


# Combine magnitude images

In [18]:
for session_dir in session_dirs:
    mag_images = sorted(glob.glob(os.path.join(session_dir, "anat", "*run-01*part-mag*nii*")))
    nii = nib.load(mag_images[0])
    mag_4d = np.array([nib.load(mag_images[i]).get_fdata() for i in range(len(mag_images))])
    mag_combined = np.sqrt(np.sum(np.square(mag_4d), axis=0))
    filename = os.path.join(session_dir, "extra_data", "magnitude_combined.nii")
    nib.save(nib.Nifti1Image(mag_combined, header=nii.header, affine=nii.affine), filename)
    print(f"Saved {filename}")

Saved bids/sub-z0002292/ses-20240318/extra_data/magnitude_combined.nii
Saved bids/sub-z0163277/ses-20240123/extra_data/magnitude_combined.nii
Saved bids/sub-z0206923/ses-20231003/extra_data/magnitude_combined.nii
Saved bids/sub-z0232771/ses-20240228/extra_data/magnitude_combined.nii
Saved bids/sub-z0347933/ses-20240129/extra_data/magnitude_combined.nii
Saved bids/sub-z0381949/ses-20231005/extra_data/magnitude_combined.nii
Saved bids/sub-z0742379/ses-20240124/extra_data/magnitude_combined.nii
Saved bids/sub-z0747844/ses-20240304/extra_data/magnitude_combined.nii
Saved bids/sub-z1318033/ses-20230825/extra_data/magnitude_combined.nii
Saved bids/sub-z1384048/ses-20231108/extra_data/magnitude_combined.nii
Saved bids/sub-z1421134/ses-20231205/extra_data/magnitude_combined.nii
Saved bids/sub-z1448271/ses-20231110/extra_data/magnitude_combined.nii
Saved bids/sub-z1484547/ses-20240102/extra_data/magnitude_combined.nii
Saved bids/sub-z1635498/ses-20240305/extra_data/magnitude_combined.nii
Saved 

# Combine QSM SNR-weighted

This stuff is old and probably not needed anymore.

In [ ]:

for session_dir in session_dirs:
    subject_id = session_dir.split(os.sep)[1]
    session_id = session_dir.split(os.sep)[2]
    print(subject_id, session_id)
    qsm_pattern = os.path.join('qsm', 'workflow', f'{subject_id}*', session_id, 'qsmxt*', 'numpy_nibabel_twopass', 'mapflow', '*', '*.nii*')
    qsm_files = sorted(glob.glob(qsm_pattern))
    qsm_files = [qsm_files[i] for i in range(len(qsm_files)) if i % 2 == 1]
    mag_pattern = os.path.join('bids-new', subject_id, session_id, 'anat', '*part-mag*nii*')
    mag_files = sorted(glob.glob(mag_pattern))
    mag_files = [mag_files[i] for i in range(len(mag_files)) if i % 2 == 1]

    qsm_images = [nib.load(f) for f in qsm_files]
    magnitude_images = [nib.load(f) for f in mag_files]

    # 2. Weights 1
    epsilon = 1e-3  # Small constant to avoid division by very small numbers
    weights = [1 / (img.get_fdata() + epsilon) for img in magnitude_images]
    sum_weights = np.sum(np.stack(weights, axis=-1), axis=-1)
    normalized_weights = [w / sum_weights for w in weights]

    # 2. Weights 2
    #weights = [1 / np.mean(img.get_fdata()) for img in magnitude_images]
    #normalized_weights = weights / np.sum(weights)
    #weighted_qsm_data = np.zeros_like(qsm_images[0].get_fdata())

    # 3. Calculate the weighted average of the QSMs:
    weighted_qsm_data = np.zeros_like(qsm_images[0].get_fdata())
    for i, qsm_img in enumerate(qsm_images):
        weighted_qsm_data += normalized_weights[i] * qsm_img.get_fdata()

    # Create a new NIfTI image with the weighted QSM data
    weighted_qsm_img = nib.Nifti1Image(weighted_qsm_data, qsm_images[0].affine, qsm_images[0].header)

    # 4. Save the resulting weighted QSM:
    nib.save(weighted_qsm_img, f"{subject_id}_{session_id}_weighted_qsm.nii.gz")
    print("1")

# Reslice T1w images

I don't think we need this section anymore because in the new data I actually transformed T1 and resliced using ITK-SNAP.

In [ ]:
for i in range(len(t1_files)):
    print(t1_files[i])
    t1_nii = nib.load(t1_files[i])
    mag_nii = nib.load(mag_files[i])
    t1_resampled = nilearn.image.resample_img(t1_nii, target_affine=mag_nii.affine, target_shape=mag_nii.header.get_data_shape())
    filename = t1_files[i].replace(".nii", "_resampled.nii")
    nib.save(t1_resampled, filename)

# Downsampling

I may not need this anymore. I only wrote it to test the 2023 models on the new dataset. The downsampling was necessary because the old models were trained on lower-resolution images.

In [ ]:
from nilearn.image import resample_img
import nibabel as nib

for i in range(len(ct_files)):
    # Load the images
    ct_img = nib.load(ct_files[i])
    seg_img = nib.load(ct_seg_clean_files[i])

    # Original image dimensions and voxel sizes
    original_shape = ct_img.shape
    original_voxel_sizes = np.diag(ct_img.affine)[:3]
    
    # Target voxel size
    target_voxel_size = 1.4
    
    # Calculate target shape based on the original field of view and target voxel size
    field_of_view = original_shape * original_voxel_sizes
    target_shape = np.round(field_of_view / target_voxel_size).astype(int)
    
    # Adjust the affine matrix to the target voxel size
    new_affine = np.copy(ct_img.affine)
    np.fill_diagonal(new_affine, [target_voxel_size, target_voxel_size, target_voxel_size, 1])

    # Resample the images
    resampled_ct_img = resample_img(ct_img, target_affine=new_affine, target_shape=target_shape, interpolation='continuous')
    resampled_seg_img = resample_img(seg_img, target_affine=new_affine, target_shape=target_shape, interpolation='nearest')

    # Save the resampled image
    resampled_ct_file = ct_files[i].replace('.nii', '_downsampled.nii')
    resampled_seg_file = ct_seg_clean_files[i].replace('.nii', '_downsampled.nii')
    
    nib.save(resampled_ct_img, resampled_ct_file)
    nib.save(resampled_seg_img, resampled_seg_file)
    print(f"Saved {resampled_ct_file}")
    print(f"Saved {resampled_seg_file}")

# Generate DICOMs for Radiographers

In [17]:
for session_dir in session_dirs[:4]:
    subject_id = session_dir.split(os.sep)[1]
    
    B0_file = [B0_file for B0_file in fmap_files if subject_id in B0_file][0]
    qsm_file = [qsm_file for qsm_file in qsm_files if subject_id in qsm_file][0]
    mag_file = [mag_file for mag_file in mag_files if subject_id in mag_file][0]
    
    !mkdir -p {os.path.join(session_dir, "extra_data", "B0_dcm")}
    !mkdir -p {os.path.join(session_dir, "extra_data", "magnitude_combined_dcm")}
    !mkdir -p {os.path.join(session_dir, "extra_data", "qsm_dcm")}
    
    !nii2dcm --dicom_type MR --centered {B0_file} {os.path.join(session_dir, "extra_data", "B0_dcm")}
    !nii2dcm --dicom_type MR {mag_file} {os.path.join(session_dir, "extra_data", "magnitude_combined_dcm")}
    !nii2dcm --dicom_type MR --centered {qsm_file} {os.path.join(session_dir, "extra_data", "qsm_dcm")}

    !tar cf {subject_id}.tar {os.path.join(session_dir, "extra_data", "B0_dcm")} {os.path.join(session_dir, "extra_data", "magnitude_combined_dcm")} {os.path.join(session_dir, "extra_data", "qsm_dcm")}

    !rm -rf {os.path.join(session_dir, "extra_data", "B0_dcm")} {os.path.join(session_dir, "extra_data", "magnitude_combined_dcm")} {os.path.join(session_dir, "extra_data", "qsm_dcm")}

nii2dcm: writing DICOM files ...
/home/uqaste15/miniconda3/envs/prostate/lib/python3.8/site-packages/pydicom/valuerep.py:290: UserWarning: The value length (19) exceeds the maximum length of 16 allowed for VR DS. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/home/uqaste15/miniconda3/envs/prostate/lib/python3.8/site-packages/pydicom/valuerep.py:290: UserWarning: The value length (18) exceeds the maximum length of 16 allowed for VR DS. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/home/uqaste15/miniconda3/envs/prostate/lib/python3.8/site-packages/pydicom/valuerep.py:290: UserWarning: The value length (20) exceeds the maximum length of 16 allowed for VR DS. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warni